# Statistical Analysis
## Descriptive, Inferential, and Exploratory Statistical Analysis

This notebook performs comprehensive statistical analysis including:
- Descriptive Statistics
- Inferential Statistics
- Exploratory Statistical Analysis


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest, shapiro, kstest
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

df = pd.read_csv('../../data/FuelConsumption.csv')
df.columns = df.columns.str.strip()


## 1. Descriptive Statistics


In [ ]:
# Measures of Central Tendency
numerical_cols = ['ENGINE SIZE', 'CYLINDERS', 'FUEL CONSUMPTION', 'COEMISSIONS']
descriptive_stats = pd.DataFrame({
    'Mean': df[numerical_cols].mean(),
    'Median': df[numerical_cols].median(),
    'Mode': [df[col].mode()[0] if len(df[col].mode()) > 0 else None for col in numerical_cols],
    'Std': df[numerical_cols].std(),
    'Variance': df[numerical_cols].var(),
    'Min': df[numerical_cols].min(),
    'Max': df[numerical_cols].max(),
    'Range': df[numerical_cols].max() - df[numerical_cols].min(),
    'Skewness': df[numerical_cols].skew(),
    'Kurtosis': df[numerical_cols].kurtosis(),
    'CV': (df[numerical_cols].std() / df[numerical_cols].mean()) * 100  # Coefficient of Variation
})
print("Descriptive Statistics:")
print(descriptive_stats.round(3))


In [ ]:
# Quartiles and IQR
print("\nQuartiles and IQR:")
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q2 = df[col].quantile(0.50)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    print(f"\n{col}:")
    print(f"  Q1: {Q1:.2f}, Q2: {Q2:.2f}, Q3: {Q3:.2f}")
    print(f"  IQR: {IQR:.2f}")


## 2. Inferential Statistics


In [ ]:
# Normality Tests
print("Normality Tests (Shapiro-Wilk for sample size < 5000, D'Agostino for larger):\n")
numerical_cols = ['ENGINE SIZE', 'CYLINDERS', 'FUEL CONSUMPTION', 'COEMISSIONS']
for col in numerical_cols:
    data = df[col].dropna()
    if len(data) < 5000:
        stat, p_value = shapiro(data)
        test_name = "Shapiro-Wilk"
    else:
        stat, p_value = normaltest(data)
        test_name = "D'Agostino"
    
    print(f"{col} ({test_name}):")
    print(f"  Statistic: {stat:.4f}, p-value: {p_value:.4f}")
    print(f"  Normal: {'Yes' if p_value > 0.05 else 'No'}\n")


In [ ]:
# T-tests: Compare fuel consumption between different groups
# Example: Compare fuel consumption by fuel type
fuel_types = df['FUEL'].unique()
if len(fuel_types) == 2:
    group1 = df[df['FUEL'] == fuel_types[0]]['FUEL CONSUMPTION']
    group2 = df[df['FUEL'] == fuel_types[1]]['FUEL CONSUMPTION']
    t_stat, p_value = stats.ttest_ind(group1, group2)
    print(f"T-test: Fuel Consumption by Fuel Type")
    print(f"  T-statistic: {t_stat:.4f}")
    print(f"  P-value: {p_value:.4f}")
    print(f"  Significant difference: {'Yes' if p_value < 0.05 else 'No'}\n")


In [ ]:
# ANOVA: Compare fuel consumption across vehicle classes
vehicle_classes = df['VEHICLE CLASS'].unique()[:5]  # Top 5 for example
groups = [df[df['VEHICLE CLASS'] == vc]['FUEL CONSUMPTION'] for vc in vehicle_classes]
f_stat, p_value = stats.f_oneway(*groups)
print(f"ANOVA: Fuel Consumption by Vehicle Class (Top 5)")
print(f"  F-statistic: {f_stat:.4f}")
print(f"  P-value: {p_value:.4f}")
print(f"  Significant difference: {'Yes' if p_value < 0.05 else 'No'}\n")


## 3. Exploratory Statistical Analysis


In [ ]:
# Confidence Intervals
print("95% Confidence Intervals for Mean:\n")
for col in numerical_cols:
    data = df[col].dropna()
    mean = data.mean()
    std = data.std()
    n = len(data)
    ci = stats.t.interval(0.95, n-1, loc=mean, scale=stats.sem(data))
    print(f"{col}:")
    print(f"  Mean: {mean:.2f}")
    print(f"  95% CI: [{ci[0]:.2f}, {ci[1]:.2f}]\n")


In [ ]:
# Correlation Analysis with p-values
from scipy.stats import pearsonr, spearmanr

print("Correlation Analysis with Significance:\n")
target = 'COEMISSIONS'
for col in ['ENGINE SIZE', 'CYLINDERS', 'FUEL CONSUMPTION']:
    data = df[[col, target]].dropna()
    pearson_corr, pearson_p = pearsonr(data[col], data[target])
    spearman_corr, spearman_p = spearmanr(data[col], data[target])
    print(f"{col} vs {target}:")
    print(f"  Pearson: r={pearson_corr:.4f}, p={pearson_p:.4f}")
    print(f"  Spearman: ρ={spearman_corr:.4f}, p={spearman_p:.4f}")
    print(f"  Significant: {'Yes' if pearson_p < 0.05 else 'No'}\n")
